# Exporting parts of the database to portable formats

Users may not always want, or have the means, to setup a fully fledged PostGIS enabled server. For the sake of facilitating reproducibility, geoslurp allows exporting parts of the database to portable file-based formats such as [geopackage](http://www.geopackage.org/). This functionality, available since version 1.1.0, can be used according to the python examples below.

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from geoslurp.config import setInfoLevel
from geoslurp.db import geoslurpConnect
setInfoLevel()

gpcon=geoslurpConnect() # this will be a connection based on the readonly user

## Exporting a subset of the PSMSL table
Here we export a query which extracts a subset (i.e. only stations within a certain polygon) of the tide gauge data from the permanent service of mean sea level (PSMSL). The resulting geopackage contains 2 layers.

In [7]:
from geoslurp.db.exporter import exportGeoQuery
from geoslurp.discover.oceanobs.psmsl import psmslQuery


import os
wkt="Polygon ((-0.91442683789107093 57.69161380668427341, 6.80867139814095879 60.46400804525987382, 10.76923459610610223 51.94879716963480121, -3.48879291656842838 50.95865637014351535, -0.91442683789107093 57.69161380668427341))"
geopout="/tmp/PSMSL_subset.gpkg"
try:
    os.remove(geopout)
except FileNotFoundError:
    pass

#export to queries as layers
exportGeoQuery(psmslQuery(gpcon,"psmsl_rlr_monthly",wkt),geopout,layer="tgm")
exportGeoQuery(psmslQuery(gpcon,"psmsl_rlr_annual",wkt),geopout,layer="tga")



## Exporting an Argo query and the profile files in the South China Sea for the first 2 weeks of December 2010



In [8]:
from geoslurp.discover.oceanobs.argoQuery import argoQuery
from datetime import datetime

southchinaDeepWKT="Polygon ((111.16091417910448058 13.48740671641791522, 113.98180970149255131 18.24766791044776326, 117.59608208955224029 20.3633395522388092, 119.97621268656716609 21.50932835820895761, 119.62360074626866435 16.92537313432836044, 119.88805970149255131 13.75186567164179507, 119.53544776119403537 12.6940298507462721, 116.36194029850747711 11.45988805970149826, 112.8358208955223887 10.40205223880597174, 110.54384328358209189 10.22574626865672087, 111.16091417910448058 13.48740671641791522))"

tspan=[datetime(2010,12,1),datetime(2010,12,15)]

exportGeoQuery(argoQuery(gpcon,geoWKT=southchinaDeepWKT,tspan=tspan)
               ,"argoSouthChinaDec2010.gpkg",layer='argo',packFiles=True)



# Exporting a selection of GRACE datasets and auxiliary datasets (without geographical information) to a local sqlite database.


In [31]:
from geoslurp.db.exporter import exportQuery
from geoslurp.discover.generic import regexQuery
outdb="itsg2018_sample.db"
try:
    os.remove(outdb)
except FileNotFoundError:
    pass

#export GRACE monthly solutions
exportQuery(regexQuery(gpcon,"itsg_grace2018_monthly_n60",scheme="gravity",uri='n60_2003')
            ,"itsg2018_sample.db",layer="grace",packFiles=True)

#export modelled ocean bottom pressure for restoring
exportQuery(regexQuery(gpcon,"itsg_grace2018_monthly_background",scheme="gravity",uri='oceanBottomPressure.+2003')
            ,"itsg2018_sample.db",layer="obp",packFiles=True)

#export a  static gravity model
exportQuery(regexQuery(gpcon,"icgem_static",scheme="gravity",uri='GOCO06s')
            ,"itsg2018_sample.db",layer="static",packFiles=True)

#export degree 1 corrections
exportQuery(regexQuery(gpcon,"geocenter_csrrl06_tn13",scheme="gravity",tspan=[datetime(2003,1,1),datetime(2003,12,31)])
            ,"itsg2018_sample.db",layer="deg1")

#also export load Love numbers
exportQuery(regexQuery(gpcon,"loadlove",scheme="earthmodels",name='PREM$'),"itsg2018_sample.db",layer="loadlove",packFiles=True)


